# Exploratory Data Analysis
We will move our exploratory analysis in this separate notebook, 
in order to remove a lot of code cluter from our main notebook.

After we have cleaned our data and they are in a format that we are confortable with, we will save them, 
and access them from our main notebook, where we will continue will working on it.

In [21]:
# imports
import pandas as pd
import numpy as np

from pandasql import sqldf

We import the library `pandasql` so that we can run SQL-style queries on our dataframes.

We will then write a lambda function that will make it easier and quicker to write our queries.
Normally, we'd have to pass in the global variables everytime we use an object. 
So in order to avoid doing this everytime, we write the lambda below to help with this.

In [22]:
# lambda function to help with the global functions need by pandasql
pysqldf = lambda q:sqldf(q, globals())

From our data, we will be using four datasets:
* imdb.title.basic located at `/data/title.basics.csv`
* imdb.title.crew located at `/data/title.crew.csv`
* imdb.title.ratings located at `/data/title.ratings.csv`
* imdb.tn.movie_budgets located at `/data/tn.movie_budgets.csv`

In [3]:
# loading our data
title_basics = pd.read_csv('./data/title.basics.csv')
title_crew = pd.read_csv('./data/title.crew.csv')
title_ratings = pd.read_csv('./data/title.ratings.csv')
movie_budgets = pd.read_csv('./data/tn.movie_budgets.csv')

Take a look at the data we have loaded to get an idea of the kind of data we will be working with.

In [4]:
#title basic top 5
title_basics.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [5]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


* Our dataset has 146144 entries
* `original_title`, `runtime_minutes`, `genres` has a few missing values
* `start_year` is in the form of integers as expected.

We will drop the `original_title` since we will not be using it and the `primary_title`, has the same content as it.

In [9]:
# take a look at the year column to see the values contained within 
title_basics.start_year.value_counts()

2017    17504
2016    17272
2018    16849
2015    16243
2014    15589
2013    14709
2012    13787
2011    12900
2010    11849
2019     8379
2020      937
2021       83
2022       32
2023        5
2024        2
2027        1
2026        1
2025        1
2115        1
Name: start_year, dtype: int64

We will also drop the entries with years beyond the current year 2022.

In [41]:
# create a copy of title basics that we can change without affecting the base table
# and drop the original_title column
title_basics_cleaned = title_basics.drop('original_title', axis=1).copy()

# drop any row that has an empty/NaN cell
title_basics_cleaned = title_basics_cleaned.dropna(axis=0, how='any')

# drop all rows in the year column that have year greater than 2022
title_basics_cleaned.drop(title_basics_cleaned[title_basics_cleaned['start_year'] > 2022].index, inplace = True)

title_basics_cleaned.info()

,tconst,primary_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,2017,116.0,NaN
